In [2]:
import os
import spacy
import pinecone
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from tqdm.autonotebook import tqdm
from sentence_transformers import SentenceTransformer

With tranformer model embeddings

In [12]:
class Solution:
    def __init__(self,model_name,embeddings) -> None:
        self.model_name=model_name
        self.embeddings=embeddings
        
    def definemodel(self):
        model = SentenceTransformer(self.model_name, device='cpu') 
        return model

    def load_file(self,filepath):
        os.chdir(("./"))
        file_loader = PyPDFLoader(os.path.join(os.getcwd(),filepath))
        documents = file_loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=30)
        chunks = text_splitter.split_documents(documents)
        print(len(chunks))
        return chunks

    def create_embed(self,model,chunks):
        for i in range(len(chunks)):
            chunk_embedding=model.encode(chunks[i].page_content).tolist()
            if len(chunk_embedding)!=0:
                self.embeddings['id'].append(i)
                self.embeddings['embed'].append(list(chunk_embedding))
        print(len(self.embeddings['id']))

    def vectordb(self,key,chunks):
        import os
        from pinecone import Pinecone as pc
        p = pc(api_key=key)
        index=p.Index("chatbot")
        print(index.describe_index_stats())
        for i in range(len(chunks)):
            index.upsert(vectors=[{
            "id": str(self.embeddings["id"][i]),
            "values": self.embeddings["embed"][i]
            }],
            namespace="data_ml")
        print(index.describe_index_stats())
        return index
    def askquestion(self, ques, index, model,chunks):
        
        query_embedding = model.encode(ques).tolist()
        xc = index.query(vector=query_embedding, top_k=3,namespace='data_ml')
        for i in xc['matches']:
            if i['score']<0.3:
                print("I don't know the answer")
                break
            else:
                doc_id=i['id']
                print(chunks[int(doc_id)])  

In [1]:

ques="For what all tasks, ML algorithms are useful?"

sol=Solution(model_name,embeddings)
model=sol.definemodel()
chunks=sol.load_file(filepath)
sol.create_embed(model,chunks)
index=sol.vectordb(key, chunks)
sol.askquestion(ques,index,model,chunks)













NameError: name 'Solution' is not defined

In [ ]:

##ALL three contain relevant answers

page_content='communication.  ML is a component of AI although it endeavors to solve the problems based on \nhistorical or previous examples. Unlike AI applications, ML involves learning of hidden patterns \nwithin the data (data mining) and subsequently using the patterns to classify or pre dict an event \nrelated to the problem . Simply  put, intelligent machines depend on the knowledge to sustain their \nfunctionalities and  ML offers the same . In essence, ML algorit hms are embedded into machines and \ndata streams provided so that knowledge and information are extracted and fed into the system for \nfaster and ef ficient management of processes (Ali et. al., 2020) .' metadata={'source': 'C:\\muskan\\IIITA\\Deep learning\\Chatbot project-Questionpro\\A Survey of Machine Learning Algorithms for 6g wirless netwok.pdf', 'page': 2}
page_content='as, data security, optimization, power -efficiency , cost-efficiency, etc. Following are the ML \nalgorithms which h elp in addressing all th

In [ ]:
question="How federated learning is better than machine learning?"
query_embedding = model.encode(question).tolist()
# query_embedding=list(map(float, query_embedding))
xc = index.query(vector=query_embedding, top_k=3, namespace='data_ml')
xc

{'matches': [{'id': '7', 'score': 0.617339313, 'values': []},
             {'id': '30', 'score': 0.599694, 'values': []},
             {'id': '67', 'score': 0.533954561, 'values': []}],
 'namespace': 'data_ml',
 'usage': {'read_units': 5}}

In [ ]:
for i in xc['matches']:
    doc_id=i['id']
    print(chunks[int(doc_id)])
##Second one contains the exact answer.

page_content='algorithms work in three different versions viz., supervised learning, unsupervised learning, and \nreinforcement learning. Few ML models such as, non -parametric Baye sian methods (Gaussian \napproach), are promising, especially in handling small, incrementally growing data sets; however, \nthey have increased complexity compared to the parametric methods. Further, the Kernel Hilbert \nSpace -based solutions have shown encoura ging results in generating improved data rate, which is \n10-100 times higher in comparison to the ones shown in the 5G wireless networks, simultaneously \nbeing computationally simple and scalable with lower approximation error. Federated Learning (FL)' metadata={'source': 'C:\\muskan\\IIITA\\Deep learning\\Chatbot project-Questionpro\\A Survey of Machine Learning Algorithms for 6g wirless netwok.pdf', 'page': 1}
page_content='8. Safe Reinforcement Learning  \nSafe R L is the process of learning policies that maximize the expectation of the return